In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# import libraries
import seaborn as sns

In [3]:
gs = pd.read_csv("../input/titanic/gender_submission.csv")
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/train.csv")


In [4]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.shape

(891, 12)

In [6]:
train.drop(columns=['PassengerId','Name','Ticket'],inplace=True)

In [7]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [8]:
train.drop(columns='Cabin',inplace=True)

In [9]:
train[train['Age'].isnull()].groupby(by='Pclass').mean()

,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,
1,0.466667,NaN,0.200000,0.033333,60.551940
2,0.363636,NaN,0.000000,0.000000,7.932955
3,0.250000,NaN,0.691176,0.227941,14.840041


In [10]:
mean_value = train.groupby(by=['SibSp','Pclass','Parch'])[['Age']].mean()
mean_value.tail()

Age
SibSp Pclass Parch           
3     3      2       6.250000
4     3      1       6.777778
             2       7.333333
5     3      2      10.200000
8     3      2            NaN

In [11]:
mean_value.loc[(0,1,0),]

Age    41.005814
Name: (0, 1, 0), dtype: float64

In [12]:
train_nan = train[train['Age'].isnull()].groupby(by=['SibSp','Pclass','Parch'])[['Age']].mean()
train_nan.tail()

Age
SibSp Pclass Parch     
1     3      1      NaN
             2      NaN
2     3      0      NaN
3     3      1      NaN
8     3      2      NaN

In [13]:
for row_index, group in train_nan.iterrows():
    for i, record in train[train['Age'].isnull()].iterrows():
        if(record['SibSp']==row_index[0] and record['Pclass']==row_index[1] and record['Parch']==row_index[2]):
            train.at[i,'Age'] = mean_value.loc[row_index,]

In [14]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         7
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [15]:
train_nan = train[train['Age'].isnull()].groupby(by=['SibSp','Pclass','Parch'])[['Age']].mean()
train_nan.head()

,,,Age
SibSp,Pclass,Parch,
8,3,2,NaN


In [16]:
for row_index, group in train_nan.iterrows():
    for i, record in train[train['Age'].isnull()].iterrows():
        if(record['Pclass']==row_index[1] and record['Parch']==row_index[2]):
            train.at[i,'Age'] = mean_value.loc[row_index,]

In [17]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         7
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [18]:
train.dropna(inplace=True)
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [19]:
train_dummies = pd.get_dummies(train[['Sex','Embarked']])
train = pd.concat([train,train_dummies],axis=1)
train.drop(columns=['Sex','Embarked'],inplace=True)
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
